In [13]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from algo import pitome

# Load a pretrained model, can be bert or distilbert .
model_ckt = 'JiaqiLee/imdb-finetuned-bert-base-uncased'
# model_ckt = 'bert-base-uncased'
# model_ckt = 'distilbert-base-uncased'
model =  AutoModelForSequenceClassification.from_pretrained(model_ckt)
tokenizer =  AutoTokenizer.from_pretrained(model_ckt)



/media/caduser/da936c0b-edd7-470e-ab92-9b972b220fe7/chau/miniconda3/envs/pitome/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/media/caduser/da936c0b-edd7-470e-ab92-9b972b220fe7/chau/miniconda3/envs/pitome/lib/python3.10/site-packages/transformers/modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arb

In [14]:
from ml_collections import ConfigDict
from tasks.tc.dataset import ImdbDataset
from tasks.tc.config import get_text_classification_config

task = ConfigDict(dict(dataset_fn=ImdbDataset, config_getter=get_text_classification_config))
config, _ = task.config_getter()    

In [15]:

path = '/media/caduser/da936c0b-edd7-470e-ab92-9b972b220fe7/chau/PiToMe/data/tc'
imdb_eval = task.dataset_fn(config, split='eval', data_path=path)    


In [16]:
import random
idx= random.randint(0, len(imdb_eval) - 1)
content, label = imdb_eval[idx]
print('label:',label)
print('='*100)
print(content)

label: tensor([1])
Excellent introspective / interpersonal piece that really had some teeth to it without feeling hopeless or worse, manipulative & artificially gratifying. Might be a good double feature with American Beauty as well. Best performance to date that I've seen from Anita Mui, and every actor in this seems like a powerhouse. Hats off to Ann Hui for the direction and Ivy Ho for the brilliant script. Seriously one of the best dramas I've seen in a while, especially if you have a taste for classical literature ALA poetry. Again, excellent.


In [17]:
inputs = tokenizer([content], truncation=True,max_length=512, padding=True, return_tensors='pt')
outputs = model(**inputs, return_dict=False)
label = outputs[0].argmax(-1).item()
print('predicted label:',label)

predicted label: 1


In [31]:
# Patch the blip's visual encoder with PiToMe.
pitome.patch.bert(model.bert.encoder)
# pitome.patch.distilbert(model.distilbert.transformer)

# Set the number of ratio of remaining token per layer. See paper for details.
model.bert.encoder.ratio = 0.9
# model.distilbert.transformer.ratio = self.ratio 

using pitome


In [32]:
outputs = model(**inputs, return_dict=False)
label = outputs[0].argmax(-1).item()
print('predicted label:',label)

predicted label: 1
